In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, random

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, Activation

In [2]:
train_x = []
train_y = []

barss = os.listdir('./bar2singlelabel_0111_/')
random.shuffle(barss)
for i in barss:
    tmp = pd.read_csv('./bar2singlelabel_0111_/' + i, compression='gzip').to_numpy()
    # if tmp[-1][0] != 1:
    train_x.append(tmp[:-1])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [10]:
train_x.shape

(9755, 31, 29)

In [11]:
df = pd.DataFrame(train_y)

s=0

for i in range(29):
    print(df[df[i] > 0].shape[0])
    s += df[df[i] > 0].shape[0]
    


547
40
905
17
42
6
165
182
1490
16
14
24
99
3
96
55
144
268
1520
255
198
6
44
33
12
835
20
88
188


In [12]:
from tensorflow.keras.layers import Bidirectional, Activation

model = tf.keras.models.Sequential()
# forward_layer = LSTM(256, dropout=0.2, recurrent_dropout=0.2)
# backward_layer = LSTM(256, activation='relu', go_backwards=True, dropout=0.2, recurrent_dropout=0.2)
# model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
# model.add(Dense(22, activation='softmax'))
model.add(tf.keras.layers.Input(shape=(31,29)))
# model.add(Bidirectional(LSTM(1024, return_sequences=True)))
model.add(Bidirectional(GRU(256, return_sequences=True)))#, dropout=0.1 , recurrent_dropout=0.1)))
# model.add(Bidirectional(GRU(256, return_sequences=True)))
model.add(Bidirectional(GRU(64, return_sequences=True)))#, dropout=0.1, recurrent_dropout=0.1)))
model.add(Bidirectional(GRU(32)))##, dropout=0.1)))
model.add(Dense(29, activation='softmax'))
# model.add(Activation('softmax'))


In [13]:
chkpnt_pth = './chkpnt/0111____/cp-{epoch:04d}.ckpt'
chkpnt_dir = os.path.dirname(chkpnt_pth)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=chkpnt_pth, save_weights_only=True, verbose=1)

In [14]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)

In [15]:
with tf.device("/gpu:0"):
    model.fit(train_x,train_y, epochs=4000, callbacks=[callback, cp_callback], validation_split=0.1, verbose=2)

Epoch 1/4000
275/275 - 8s - loss: 1.6946 - accuracy: 0.2644 - val_loss: 1.6790 - val_accuracy: 0.2777

Epoch 00001: saving model to ./chkpnt/0111____\cp-0001.ckpt
Epoch 2/4000
275/275 - 3s - loss: 1.6098 - accuracy: 0.2599 - val_loss: 1.6034 - val_accuracy: 0.2500

Epoch 00002: saving model to ./chkpnt/0111____\cp-0002.ckpt
Epoch 3/4000
275/275 - 3s - loss: 1.5835 - accuracy: 0.2556 - val_loss: 1.6281 - val_accuracy: 0.2449

Epoch 00003: saving model to ./chkpnt/0111____\cp-0003.ckpt
Epoch 4/4000
275/275 - 3s - loss: 1.5719 - accuracy: 0.2588 - val_loss: 1.6090 - val_accuracy: 0.2736

Epoch 00004: saving model to ./chkpnt/0111____\cp-0004.ckpt
Epoch 5/4000
275/275 - 3s - loss: 1.5561 - accuracy: 0.2851 - val_loss: 1.5537 - val_accuracy: 0.3371

Epoch 00005: saving model to ./chkpnt/0111____\cp-0005.ckpt
Epoch 6/4000
275/275 - 3s - loss: 1.5453 - accuracy: 0.2860 - val_loss: 1.5335 - val_accuracy: 0.3145

Epoch 00006: saving model to ./chkpnt/0111____\cp-0006.ckpt
Epoch 7/4000
275/275 -

In [19]:
model.save('gru_1230.h5')

In [21]:
import functools
top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

In [342]:
encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=64, return_state=True, dropout=0.2, recurrent_dropout=0.2)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [343]:
decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=64, return_state=True, dropout=0.2, recurrent_dropout=0.2)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [86]:
from tensorflow.keras.regularizers import l2

In [386]:
units = 256
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [387]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [388]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [363]:
dec = np.zeros_like(train_x)

for i in range(dec.shape[0]):
    for j in range(dec.shape[1]):
        if j != 0:
            dec[i][j] = train_x[i][j-1]

In [389]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
169/169 [==============================] - 114s 658ms/step - loss: 3.0310 - accuracy: 0.3491 - val_loss: 2.1665 - val_accuracy: 0.3900
Epoch 2/5000
169/169 [==============================] - 112s 662ms/step - loss: 2.0563 - accuracy: 0.3921 - val_loss: 2.0777 - val_accuracy: 0.4000
Epoch 3/5000
169/169 [==============================] - 111s 657ms/step - loss: 2.0036 - accuracy: 0.3934 - val_loss: 2.0188 - val_accuracy: 0.4067
Epoch 4/5000
169/169 [==============================] - 112s 661ms/step - loss: 1.9704 - accuracy: 0.3916 - val_loss: 1.9819 - val_accuracy: 0.3967
Epoch 5/5000
169/169 [==============================] - 113s 667ms/step - loss: 1.9567 - accuracy: 0.3949 - val_loss: 1.9972 - val_accuracy: 0.4033
Epoch 6/5000
169/169 [==============================] - 112s 660ms/step - loss: 1.9412 - accuracy: 0.3866 - val_loss: 1.9683 - val_accuracy: 0.4133
Epoch 7/5000
169/169 [==============================] - 111s 654ms/step - loss: 1.9285 - accuracy: 0.3980 - val_

In [49]:
model.predict(np.expand_dims(train_x[0],axis=0))

array([[6.2003901e-04, 2.8944915e-04, 7.9551857e-05, 1.8357024e-02,
        1.5380255e-04, 2.6598177e-06, 3.5440738e-05, 1.2881367e-03,
        5.1815554e-05, 9.0529507e-01, 3.5265017e-05, 5.8755501e-05,
        3.2746611e-05, 9.9628800e-05, 5.8914007e-05, 3.6296697e-04,
        8.6647044e-05, 7.0513055e-02, 1.7987693e-05, 9.9692668e-04,
        5.5625511e-04, 3.3139176e-04, 6.9041467e-05, 3.9404353e-05,
        1.9126725e-05, 2.6608466e-05, 5.1444356e-04, 2.3397490e-06,
        5.5000392e-06]], dtype=float32)

In [ ]:
model.save('ende_drp2.h5')

dilla.wav


In [8]:
# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0], # 0 null
    [1, 0, 0, 0, 0, 0, 0, 0, 0], # 1  B
    [1, 0, 0, 0, 0, 0, 0, 1, 0], # 2  B+C
    [1, 0, 0, 0, 0, 0, 1, 0, 0], # 3  B+CH
    [1, 0, 1, 0, 0, 0, 0, 0, 0], # 4  B+HT
    [1, 0, 0, 0, 1, 0, 0, 0, 0], # 5  B+LT
    [1, 0, 0, 1, 0, 0, 0, 0, 0], # 6  B+MT
    [1, 0, 0, 0, 0, 1, 0, 0, 0], # 7  B+OH
    [1, 0, 0, 0, 0, 0, 0, 0, 1], # 8  B+R
    [0, 0, 0, 0, 0, 0, 1, 0, 0], # 9  CH
    [0, 0, 1, 0, 0, 0, 1, 0, 0], # 10 CH+HT
    [0, 0, 0, 0, 1, 0, 1, 0, 0], # 11 CH+LT
    [0, 0, 0, 0, 0, 0, 1, 0, 1], # 12 CH+R
    [0, 0, 1, 0, 0, 0, 0, 0, 0], # 13 HT
    [0, 0, 1, 0, 1, 0, 0, 0, 0], # 14 HT+LT
    [0, 0, 0, 0, 1, 0, 0, 0, 0], # 15 LT
    [0, 0, 0, 1, 0, 0, 0, 0, 0], # 16 MT
    [0, 0, 0, 0, 0, 1, 0, 0, 0], # 17 OH
    [0, 0, 0, 0, 0, 0, 0, 0, 1], # 18 R
    [0, 1, 0, 0, 0, 0, 0, 0, 0], # 19 S
    [1, 1, 0, 0, 0, 0, 0, 0, 0], # 20 S+B
    [1, 1, 0, 0, 0, 0, 1, 0, 0], # 21 S+B+CH
    [1, 1, 0, 0, 1, 0, 0, 0, 0], # 22 S+B+LT
    [1, 1, 0, 0, 0, 1, 0, 0, 0], # 23 S+B+OH
    [1, 1, 0, 0, 0, 0, 0, 0, 1], # 24 S+B+R
    [0, 1, 0, 0, 0, 0, 0, 1, 0], # 25 S+C
    [0, 1, 0, 0, 0, 0, 1, 0, 0], # 26 S+CH
    [0, 1, 0, 0, 1, 0, 0, 0, 0], # 27 S+LT
    [0, 1, 0, 0, 0, 1, 0, 0, 0], # 28 S+OH
    [0, 1, 0, 0, 0, 0, 0, 0, 1], # 29 S+R
    # [0, 0, 0, 0, 0, 0, 0, 1, 0], # 30 C
]


seven_kind_arr = [
   # bd sn mt lt oh ch ri 
    [1, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0], #1
    [0, 0, 1, 0, 0, 0, 0], #2
    [0, 0, 0, 1, 0, 0, 0], #3
    [0, 0, 0, 0, 1, 0, 0], #4
    [0, 0, 0, 0, 0, 1, 0], #5
    [0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 1, 0], #7
    [1, 0, 0, 0, 1, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 1, 0, 0, 0], #10
    [1, 1, 0, 0, 0, 0, 0], #11
    [0, 1, 0, 0, 0, 1, 0], #12
    [0, 1, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 0, 1], #14
    [0, 1, 0, 1, 0, 0, 0], #15
    [0, 1, 0, 0, 0, 0, 1], #16
    [0, 0, 1, 1, 0, 0, 0], #17
    [1, 1, 0, 0, 0, 1, 0], #18
    [1, 1, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 0, 1], #20
    [0, 0, 0, 0, 0, 0, 0]
]




In [49]:
truth_loop = [7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21]

# truth_loop = [7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22]

other = [7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5]

prd = [0, 21, 21, 21, 16, 21, 21, 21, 5,  21, 0 , 21, 1 , 0,  21, 21,\
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0 , 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 0 , 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 10, 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 12, 21, 0,  21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 16, 21, 21, 21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 0,  21, 1 , 10, 21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 0,  21, 7,  21, 0,  21, 14, 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 2,  21, 21, 21, 0,  21, 1,  0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1,  0,  21, 21, \
       0, 21, 21, 21, 12, 21, 21, 21, 5,  21, 0,  21, 1,  0,  21, 2]
ground_truth = [] # loop * 12

threefour = [7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21]

asdf = [7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 5, 21, 5, 21, 2, 2, 3, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
       #  7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
       #  7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
              7, 21, 5, 21, 12, 21, 5, 21, 5, 21, 5, 21, 2, 2, 3, 21]

#1

asd = [5, 21, 7, 21, 5, 21, 12, 21, 5, 21,\
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 7, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 7, 21, 2]

# asd = [21, 7, 21, 5, 21, 12, 21, 5, 21,\
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 0,  0,  10, 21, \
#        9, 21, 21, 21, 5, 21, 21, 21, \
#        5, 21, 21, 21, 12, 21, 21, 21, \
#        7, 21 ,21, 21, 12, 21, 0, 12, \
#        5, 21, 21, 21, 12, 21, 21, 21, \
#        7, 21, 21, 21, 5, 21]

# asd = [
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 0,  0,  10, 21, \
#        9, 21, 21, 21, 5, 21, 21, 21, \
#        5, 21, 21, 21, 12, 21, 21, 21, \
#        7, 21 ,21, 21, 12, 21, 0, 12, \
#        5, 21, 21, 21, 12, 21, 21, 21, \
#        7, 21, 21, 21, 5, 21, #0, 21, \
#        # 5, 21, 21, 21, 12, 21, 21, 21, \
#        # 21, 2, 21, 2, 21, 2, 21]
# ]

# # #2
# asd = [21, 7, 21, 7, 21, 12, 21, 5, 21, \
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 12, 21, 5, 21, \
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 12, 21, 5, 21, \
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 12, 21, 5, 21, \
#        1, 1, 21, 2, 21, 3]


In [98]:
model = tf.keras.models.load_model('./0927(4).h5')

In [89]:
who2test = truth_loop
# prd_input = np.zeros((len(who2test), 22))
prd_input = []
# i = 0
# for p in who2test:
#     prd_input[i][int(p)] = 1
#     i += 1

for p in who2test:
    prd_input.append(seven_kind_arr[p])

In [99]:
for i in range(30):
    asd = model.predict(x=np.expand_dims(prd_input[i:i+31], axis=0))[0]
    # print(seven_kind_arr[np.argmax(asd)])
    print(asd.round())
# print(np.argmax(asd))
# print(asd[0].round(4))
# print(asd[10].round(4))
# print(asd[9].round(4))
# print('------')
# print(asd[2].round(4))
# asd.round(3)
# print(asd.round(1))
# seven_kind_arr[np.argmax(model.predict(x=np.expand_dims(prd_input[-31:], axis=0)))]

[0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 1. 0.]


In [ ]:
change = []
for i in range(prd_input.__len__() - 63):
    # a = model.predict(x=[np.expand_dims(prd_input[i:i+63], axis=0),np.expand_dims(testdec[i:i+63],axis=0)])
    a = model.predict(x=np.expand_dims(prd_input[i:i+63], axis=0))
    # print(a)
    change.append(np.argmax(a))
    change.append(a)
    # imsi = []
    # for i in a:
    #     imsi.append(i.round())
    # change.append(imsi)
        

In [116]:
change

[21,
 array([[5.65134594e-03, 3.94633114e-02, 3.38134123e-03, 4.42493969e-04,
         2.26508430e-03, 4.76730168e-02, 1.87292462e-04, 3.09590250e-04,
         2.24718708e-03, 1.79201961e-04, 2.69095199e-05, 4.13705653e-04,
         8.92597483e-04, 4.84599586e-05, 3.82965254e-06, 1.95931552e-05,
         7.64672222e-07, 1.33016219e-04, 1.07813554e-04, 1.39077201e-05,
         4.29085048e-05, 8.96496654e-01]], dtype=float32),
 7,
 array([[7.1322737e-04, 7.7466376e-04, 1.0791187e-04, 3.7272188e-05,
         1.7020659e-04, 2.8024239e-02, 3.6148187e-05, 9.4857550e-01,
         5.5231987e-03, 4.4115507e-03, 7.9494848e-06, 4.5447238e-04,
         5.1352084e-03, 3.8101382e-05, 6.6804097e-07, 1.9557234e-05,
         2.5317848e-07, 5.0105800e-06, 5.5752336e-03, 9.0990667e-05,
         2.4908888e-05, 2.7377796e-04]], dtype=float32),
 21,
 array([[5.5735058e-04, 1.1359632e-03, 3.2819582e-06, 2.4064342e-08,
         7.0135393e-06, 5.8192769e-03, 6.3336188e-06, 3.4540259e-05,
         3.9442151e-05

------
추석때 돌려놓고 갈 것

In [ ]:
units = 64
dropout = 0.4

model = tf.keras.models.Sequential()
forward_layer = LSTM(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = LSTM(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('bidunits64dropout0.4.h5')

In [90]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [91]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

NameError: name 'train_x' is not defined

In [ ]:
model.save('bidunits128dropout0.2l2norm.h5')

In [ ]:
units = 64
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('unit64dropout0.4.h5')

In [ ]:
units = 512
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('unit512dropout0.4.h5')

In [ ]:
units = 256
dropout = 0.1

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('unit256dropout0.1l2norm.h5')